In [70]:
# %%
import numpy as np
from numpy.fft import rfft2, irfft2, rfftfreq, fftfreq
import matplotlib.pyplot as plt
import vtk
from noises import pink_noise

np.random.seed(42)

# Define the grid dimensions
nx = 128  # Number of points in x direction
ny = 128  # Number of points in y direction

# Define the range for x and y
x_min, x_max = 0, 1  # Range for x
y_min, y_max = 0, 1  # Range for y

# Create the vertices
x_vertices = np.linspace(x_min, x_max, nx+1)  # Create x values
y_vertices = np.linspace(y_min, y_max, ny+1)  # Create y values

# Use meshgrid to create the 2D vertices grid
XV, YV = np.meshgrid(x_vertices, y_vertices)

# Create the cell centers
x_centers = (x_vertices[:-1]+x_vertices[1:]) / 2.0  # Create x values
y_centers = (y_vertices[:-1]+y_vertices[1:]) / 2.0  # Create y values

# Use meshgrid to create the 2D grid
X, Y = np.meshgrid(x_centers, y_centers)

# Calculate the step sizes in x and y directions
dx = x_centers[1] - x_centers[0]
dy = y_centers[1] - y_centers[0]

# Calculate the wavenumbers in x and y directions
kx = 2 * np.pi * rfftfreq(nx, dx)
ky = 2 * np.pi * fftfreq(ny, dy)

# Use meshgrid to create the 2D grid
KX, KY = np.meshgrid(kx, ky)
K2 = KX**2 + KY**2
K = np.sqrt(K2)
INVK2 = np.where(K2==0, 0, 1.0/K2)
KNORM = np.sqrt((KX/np.max(KX))**2 + (KY/np.max(KY))**2)
SPEC_FILTER = np.exp(-23.6 * 1024 * (KNORM - 2.0/3.0)**4.)
SPEC_FILTER[KNORM <= 2.0/3.0] = 1

/var/folders/7s/tn4_jx8158vf7rpdxlfyncr40000gn/T/ipykernel_27388/2760666368.py:44: RuntimeWarning: divide by zero encountered in divide
  INVK2 = np.where(K2==0, 0, 1.0/K2)


In [71]:
def create_vtk_image_data(souts):
    # Create vtkImageData
    imageData = vtk.vtkImageData()
    imageData.SetDimensions(nx+1, ny+1, 1)
    imageData.SetSpacing(dx, dy, 1)
    imageData.SetOrigin(x_min, y_min, 0)
    
    for key in souts:
        # Create temperature data array
        tempArray = vtk.vtkDoubleArray()
        tempArray.SetName(key)
        tempArray.SetNumberOfComponents(1)
        tempArray.SetNumberOfTuples(nx * ny)
        tempArray.SetArray(souts[key].ravel(), nx * ny, 1)
        imageData.GetCellData().SetScalars(tempArray)
    
    return imageData

def write_vtk_image_data(imageData, filename):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(imageData)
    writer.SetDataMode(vtk.vtkXMLWriter.Binary)
    writer.Write()

In [72]:
OMEGAH = rfft2(pink_noise(ny, nx))
OMEGAH *= SPEC_FILTER*(ny/100*nx/100)
PSIHAT = OMEGAH * INVK2
U = irfft2(1j*KY*PSIHAT)
V =-irfft2(1j*KX*PSIHAT)
E0 = 0.5*np.sum(U**2+V**2)

nu = 1e-6
nt = 101
t = 0
CFL = 0.5
dt = CFL/max(np.max(np.abs(U/dx)),np.max(np.abs(V/dy)))
nt_sub = int(np.ceil(100.0/nt/dt))
print(dt, nt_sub)

0.041793876427292785 24


In [73]:
def N(OMEGAH):
    PSIHAT = OMEGAH * INVK2
    NONL = -irfft2(1j*KY*PSIHAT)*irfft2(1j*KX*OMEGAH) + irfft2(1j*KX*PSIHAT)*irfft2(1j*KY*OMEGAH)
    NONL = rfft2(NONL) * SPEC_FILTER
    NONL += -nu * K2 * OMEGAH
    return NONL

for i in range(nt):
    for _ in range(nt_sub):
        k1 = dt * N(OMEGAH)
        k2 = dt * N(OMEGAH+1.0/2.0*k1)
        k3 = dt * N(OMEGAH+1.0/2.0*k2)
        k4 = dt * N(OMEGAH+k3)
        OMEGAH += 1.0/6.0*(k1 + 2*k2 + 2*k3 + k4)
        t += dt

    PSIHAT = OMEGAH * INVK2
    U = irfft2(1j*KY*PSIHAT)
    V =-irfft2(1j*KX*PSIHAT)
    OMEGA = irfft2(OMEGAH)
    E = 0.5*np.sum(U**2+V**2)
    CFL = dt*max(np.max(np.abs(U/dx)),np.max(np.abs(V/dy)))
    print(f"{i}, {t:.2f}, {CFL:.2f}, {E/E0:.4f}, {np.sum(OMEGA)}")

    imageData = create_vtk_image_data({"omega": OMEGA})
    filename = f"results/vort_transport_2d_{i:03d}.vti"
    write_vtk_image_data(imageData, filename)


0, 1.00, 0.49, 0.9980, 0.0
1, 2.01, 0.51, 0.9962, -2.2737367544323206e-13
2, 3.01, 0.48, 0.9946, -1.1368683772161603e-13
3, 4.01, 0.44, 0.9930, -1.1368683772161603e-13
4, 5.02, 0.49, 0.9916, -2.2737367544323206e-13
5, 6.02, 0.49, 0.9903, -1.1368683772161603e-13
6, 7.02, 0.47, 0.9890, -1.1368683772161603e-13
7, 8.02, 0.50, 0.9878, -3.979039320256561e-13
8, 9.03, 0.50, 0.9867, -2.8421709430404007e-13
9, 10.03, 0.49, 0.9856, -1.7053025658242404e-13
10, 11.03, 0.47, 0.9846, -5.684341886080802e-14
11, 12.04, 0.46, 0.9837, -2.2737367544323206e-13
12, 13.04, 0.45, 0.9828, -1.7053025658242404e-13
13, 14.04, 0.44, 0.9820, -1.1368683772161603e-13
14, 15.05, 0.44, 0.9812, -1.7053025658242404e-13
15, 16.05, 0.48, 0.9804, -1.1368683772161603e-13
16, 17.05, 0.48, 0.9797, -1.1368683772161603e-13
17, 18.05, 0.46, 0.9790, 5.684341886080802e-14
18, 19.06, 0.46, 0.9784, 0.0
19, 20.06, 0.47, 0.9778, -1.1368683772161603e-13
20, 21.06, 0.48, 0.9772, -5.684341886080802e-14
21, 22.07, 0.48, 0.9766, -1.1368683